# **Basics**

In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

**Menu**
- Basic autograd example 1
- Basic autograd example 2
- Loading data from numpy
- Input pipeline
- Input pipeline for custom dataset
- Pretrained model
- save and load model

### **Basic auto grad 1**

In [3]:
# create tensors
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# build a computation graph
y = w * x + b

# compute gradients
y.backward()

# Gradients
print(x.grad)  # 2
print(w.grad)  # 1
print(b.grad)  # 1

tensor(2.)
tensor(1.)
tensor(1.)


In [7]:
print(x.data)  # get tensor
print(x.grad)  # get gradient

tensor(1.)
tensor(2.)


### **Basic autograd example 2**

In [9]:
# create tensor
x = torch.rand(10, 3)
y = torch.rand(10, 2)

# build a fully connect layer
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b: ', linear.bias)

# build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass
pred = linear(x)

# Compute loss
loss = criterion(pred, y)
print('loss: ', loss.item())

# Backward pass
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()
# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[-0.4003,  0.1447, -0.3484],
        [ 0.5704, -0.1693, -0.5088]], requires_grad=True)
b:  Parameter containing:
tensor([-0.1671,  0.2010], requires_grad=True)
loss:  0.5659213066101074
dL/dw:  tensor([[-0.3239, -0.4340, -0.4360],
        [-0.1124, -0.2482, -0.3122]])
dL/db:  tensor([-0.8166, -0.4089])
loss after 1 step optimization:  0.5511578321456909


### **numpy.array & torch.tensor**

In [10]:
# create a numpy array
x = np.array([[1, 2],[3, 4]])

# convert numpy array into tensor
y = torch.from_numpy(x)

# convert the torch tensor to a numpy array
z = y.numpy()

In [11]:
x, y, z

(array([[1, 2],
        [3, 4]]),
 tensor([[1, 2],
         [3, 4]], dtype=torch.int32),
 array([[1, 2],
        [3, 4]]))

### **Input pipeline**

In [14]:
# Construct the Dataset (not in the ram)
train_dataset = torchvision.datasets.CIFAR10(root="./data/",
                                             train=True,
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)

# Construct Dataloader (provides queues and threads in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=64,
                                          shuffle=True)


# Iteration starts, queque and thread start to load data from files
data_iter = iter(train_loader)

# Mini-batch images and labels
images, labels = data_iter.next()

# Usage of Dataloader
for images, labels in train_loader:
    # Training code here
    pass

Files already downloaded and verified
torch.Size([3, 32, 32])
6


### **Custom Dataset**

In [16]:
from torch.utils.data import Dataset

In [20]:
# build your own dataset
class CustomDataset(Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names
        super(CustomDataset, self).__init__()
        pass
    
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile / PIL.Image.open
        # 2. Preprocess the data (e.g. torch.vision. transforms)
        # 3. Return a data pair (e.g. image & label)
        pass
    
    def __len__(self):
        # TODO
        # 1. return total size of your dataset
        return 0
    
# Use the Dataloader
custom_dataset = CustomDataset()
# train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
#                                           batch_size=64,
#                                           shuffle=True)

# Usage of Dataloader
# for images, labels in train_loader:

### **Pretrained Model - Finetune**

In [22]:
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False
    
# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print(outputs.size())

torch.Size([64, 100])


### **Save & Load model**

In [23]:
# save & load the entire model
torch.save(resnet, 'model/model.ckpt')
model = torch.load('model/model.ckpt')

# save and load the model parameters (recommend)
torch.save(resnet.state_dict(), 'model/params.ckpt')
resnet.load_state_dict(torch.load('model/params.ckpt'))

<All keys matched successfully>